In [1]:
class Generator:
    '''
    The Generator class is responsible for gathering historical data of various lengths. 
    
    First, there is the 'first_pull' function which gathers historical data as far back as the instrument
    is available until datetime.now(). 
    
    Second, there is an 'update' function which updates from the last entry of the first_pull until datetime.now()
    periodically. 
    
    The differences have to do with the steps involved to pass through a large database vs. a small database. 
    '''
    
    def __init__(self, instrument, gran):
        M = {"S30":{"step": 60,
                    "pstep": 21600,
                    "pull": 2,
                    "sleep": 30,
                    "idt": dt.datetime(2020, 3, 1)},
             "M1":{"step": 60,
                   "pstep": 21600*13,
                   "pull": 2,
                   "sleep": 60,
                   "idt": dt.datetime(2019, 1, 1)},
             "M5":{"step": 60,
                   "pstep": 21600*12*5,
                   "pull": 5,
                   "sleep": 60*5,
                   "idt": dt.datetime(2015, 1, 1)},
             "M10":{"step": 60,
                    "pstep": (21600*12*5)/2,
                    "pull": 10,
                    "sleep": 60*10,
                    "idt": dt.datetime(2010, 1, 1)},
             "M30":{"step": 60,
                    "pstep": (21600*12*5)/6,
                    "pull": 30,
                    "sleep": 60*30,
                    "idt": dt.datetime(2005, 1, 1)},
             "D":{"step": 60,
                  "pstep": 21000*5*12,
                  "pull": 60*24,
                  "sleep": 60*60*24,
                  "idt": dt.datetime(2010, 1, 1)}
            }
        self.instrument = instrument
        self.gran = gran
        self.step = M[gran]['step']
        self.pstep = M[gran]['pstep']
        self.pull = M[gran]['pull']
        self.sleep = M[gran]['sleep']
        self.idt = M[gran]['idt']
        
    def prep_data(self, dt1, dt2):

        begin_unix = int(dt1.replace(tzinfo=timezone.utc).timestamp())
        end_unix = int(dt2.replace(tzinfo=timezone.utc).timestamp())

        i = begin_unix + self.pstep

        dataset = pd.DataFrame()

        params = {"from": str(begin_unix),
                  "to": str(end_unix),
                  "granularity": self.gran,
                  "price": 'BA'}

        while i <= end_unix:
            params['from'] = str(i-self.pstep)
            params['to'] = str(i)
            r = instruments.InstrumentsCandles(instrument=self.instrument, params=params)
            data = client.request(r)
            results= [{"time":x['time'],
                       "open":float(x['ask']['o']),
                       "high":float(x['ask']['h']),
                       "low":float(x['ask']['l']),
                       "close":float(x['ask']['c']),
                       #"spread":(float(x['bid']['c']) - float(x['ask']['c'])),
                       #"volume":float(x['volume'])
                      } for x in data['candles']]
            df = pd.DataFrame(results)
            dataset = dataset.append(df, ignore_index=True)
            if (i + self.pstep) >= end_unix:
                params['from'] = str(i)
                params['to'] = str(end_unix)
                r = instruments.InstrumentsCandles(instrument=self.instrument, params=params)
                data = client.request(r)
                results= [{"time":x['time'],
                           "open":float(x['ask']['o']),
                           "high":float(x['ask']['h']),
                           "low":float(x['ask']['l']),
                           "close":float(x['ask']['c']),
                           #"spread":(float(x['bid']['c']) - float(x['ask']['c'])),
                          #"volume":float(x['volume'])
                          } for x in data['candles']]
                df = pd.DataFrame(results)
                i += self.pstep
                dataset = dataset.append(df, ignore_index=True)
            i += self.pstep
        dataset['time'] = dataset.time.apply(lambda x: get_date_object(x))
        dataset = dataset.set_index('time')
        return dataset
    
    def sleepytime(self):
        time.sleep(self.sleep) 
        
    def first_pull(self):
        dt2 = dt.datetime(2020, 10, 7)
        df = Generator.prep_data(self, self.idt, dt2)
        return df
    
    def update(self, init):
        df = Generator.pull(self)
        period = init + dt.timedelta(minutes=self.pull)
        df = df.loc[period:]
        return df
        
    def pull(self):
        dt1 = dt.datetime.now() - dt.timedelta(minutes=self.pull)
        dt2 = dt.datetime.now()
        df = Generator.raw_pull(self, dt1, dt2).dropna()
        return df
    
    def raw_pull(self, dt1, dt2):
        begin_unix = int(dt1.replace(tzinfo=timezone.utc).timestamp())
        end_unix = int(dt2.replace(tzinfo=timezone.utc).timestamp())

        i = begin_unix + self.step

        dataset = pd.DataFrame()

        params = {"from": str(begin_unix),
                  "to": str(end_unix),
                  "granularity": self.gran,
                  "price": 'BA'}

        while i <= end_unix:
            params['from'] = str(i-self.step)
            params['to'] = str(i)
            r = instruments.InstrumentsCandles(instrument=self.instrument, params=params)
            data = client.request(r)
            results= [{"time":x['time'],
                       "open":float(x['ask']['o']),
                       "high":float(x['ask']['h']),
                       "low":float(x['ask']['l']),
                       "close":float(x['ask']['c']),
                       #"spread":(float(x['bid']['c']) - float(x['ask']['c'])),
                       # "volume":float(x['volume'])
                      } for x in data['candles']]
            df = pd.DataFrame(results)
            dataset = dataset.append(df, ignore_index=True)
            if (i + self.step) >= end_unix:
                params['from'] = str(i)
                params['to'] = str(end_unix)
                r = instruments.InstrumentsCandles(instrument=self.instrument, params=params)
                data = client.request(r)
                results= [{"time":x['time'],
                           "open":float(x['ask']['o']),
                           "high":float(x['ask']['h']),
                           "low":float(x['ask']['l']),
                           "close":float(x['ask']['c']),
                           #"spread":(float(x['bid']['c']) - float(x['ask']['c']))
                          } for x in data['candles']]
                df = pd.DataFrame(results)
                i += self.step
                dataset = dataset.append(df, ignore_index=True)
            i += self.step
        dataset['time'] = dataset.time.apply(lambda x: get_date_object(x))
        dataset = dataset.set_index('time')
        return dataset